# 80. 畳み込みとは何か（直感編）

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

畳み込みニューラルネットワーク（CNN）を学ぶ旅の最初の一歩として、**「畳み込み」という操作の直感的な理解**を目指します。数式やコードに入る前に、この操作が私たちの日常のどこにあり、なぜ重要なのかを体感しましょう。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口
├── 80. 畳み込みとは何か（直感編）  ← 今ここ
├── 81. 畳み込みの数学的定義
├── 82. NumPyスクラッチ実装（基礎編）
├── 83. NumPyスクラッチ実装（高速化編）
├── 84. 古典的フィルタの解剖学
└── 85. カーネルの可視化と解釈
```

## 学習目標

このノートブックを終えると、以下のことができるようになります：

1. **畳み込みの本質を一言で説明できる**：「周囲を見て、自分の値を決める」操作
2. **日常の例と結びつけられる**：ぼかし、移動平均、エコーなど
3. **1次元と2次元の畳み込みの関係を理解できる**
4. **なぜ画像処理に畳み込みが使われるのか、直感的に説明できる**

## 目次

1. [日常にある「畳み込み」の例](#1-日常にある畳み込みの例)
2. [1次元信号での畳み込み入門](#2-1次元信号での畳み込み入門)
3. [2次元への拡張：画像という信号](#3-2次元への拡張画像という信号)
4. [まとめ：なぜこの操作が重要なのか](#4-まとめなぜこの操作が重要なのか)

---

## 環境セットアップ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（環境に応じて調整）
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# 再現性のためのシード
np.random.seed(42)

print("環境準備完了")

---

## 1. 日常にある「畳み込み」の例

「畳み込み」という言葉は難しく聞こえますが、実は私たちの周りにたくさんあります。

### 1.1 ぼかしガラス越しに見る世界

お風呂場のすりガラス越しに外を見たことはありますか？

**何が起きているか：**
- 1つの点から来た光が、ガラスを通過する際に周囲に広がる
- 結果として、各点の色が「周囲の色と混ざった」ように見える
- これが**ぼかし（blur）**の本質

これはまさに畳み込みです：**「周囲の情報を集めて、自分の値を決める」**

In [ ]:
# ぼかしガラスの効果をシミュレーション

def create_simple_image():
    """シンプルなテスト画像を作成"""
    img = np.zeros((100, 100))
    # 白い四角形
    img[30:70, 30:70] = 1.0
    return img

def simple_blur(image, size=5):
    """シンプルな平均ぼかし（畳み込みの一種）"""
    result = np.zeros_like(image)
    pad = size // 2
    padded = np.pad(image, pad, mode='reflect')
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # 周囲のsize x size領域の平均を取る
            neighborhood = padded[i:i+size, j:j+size]
            result[i, j] = np.mean(neighborhood)
    
    return result

# 可視化
original = create_simple_image()
blurred_3 = simple_blur(original, size=3)
blurred_7 = simple_blur(original, size=7)
blurred_15 = simple_blur(original, size=15)

fig, axes = plt.subplots(1, 4, figsize=(14, 3.5))

images = [original, blurred_3, blurred_7, blurred_15]
titles = ['元の画像', 'ぼかし（3x3）', 'ぼかし（7x7）', 'ぼかし（15x15）']

for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap='gray', vmin=0, vmax=1)
    ax.set_title(title)
    ax.axis('off')

plt.suptitle('ぼかし = 「周囲の平均を取る」畳み込み', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察ポイント：")
print("- 周囲を見る範囲（3x3, 7x7, 15x15）が大きいほど、ぼかしが強くなる")
print("- エッジ（境界）が徐々に滑らかになっていく")
print("- これが『畳み込み』の最も基本的な例です")

### 1.2 移動平均という畳み込み

株価やセンサーデータでよく使われる「移動平均」も畳み込みの一種です。

**移動平均の考え方：**
- 今日の「平滑化された値」= 過去N日間の平均
- つまり、「周囲（過去）の値を集めて、自分の値を決める」

これは1次元の畳み込みです。

In [ ]:
# 移動平均の例：ノイズの多い株価データ

# 模擬的な株価データを生成
days = 200
t = np.arange(days)

# 上昇トレンド + 周期的な変動 + ノイズ
trend = 100 + 0.3 * t
seasonal = 10 * np.sin(2 * np.pi * t / 30)
noise = np.random.randn(days) * 8
stock_price = trend + seasonal + noise

def moving_average(data, window_size):
    """移動平均を計算（畳み込みの一種）"""
    result = np.zeros_like(data)
    for i in range(len(data)):
        start = max(0, i - window_size + 1)
        result[i] = np.mean(data[start:i+1])
    return result

# 異なるウィンドウサイズで移動平均を計算
ma_5 = moving_average(stock_price, 5)
ma_20 = moving_average(stock_price, 20)
ma_50 = moving_average(stock_price, 50)

# 可視化
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# 上段：元データと移動平均
ax1 = axes[0]
ax1.plot(t, stock_price, 'gray', alpha=0.5, linewidth=0.8, label='生データ（ノイズあり）')
ax1.plot(t, ma_5, 'b-', linewidth=1.5, label='5日移動平均')
ax1.plot(t, ma_20, 'orange', linewidth=2, label='20日移動平均')
ax1.plot(t, ma_50, 'r-', linewidth=2.5, label='50日移動平均')
ax1.set_xlabel('日数')
ax1.set_ylabel('株価')
ax1.set_title('移動平均 = 「過去N日間の平均」を取る畳み込み', fontsize=13, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 下段：ウィンドウの概念図
ax2 = axes[1]
focus_start = 80
focus_end = 120
ax2.plot(t[focus_start:focus_end], stock_price[focus_start:focus_end], 'gray', 
         alpha=0.7, linewidth=1, marker='o', markersize=4, label='生データ')

# 特定の点での移動平均計算を可視化
target_day = 100
window = 7
window_start = target_day - window + 1

# ウィンドウ範囲をハイライト
ax2.axvspan(window_start, target_day, alpha=0.3, color='yellow', label=f'{window}日ウィンドウ')
ax2.scatter([target_day], [np.mean(stock_price[window_start:target_day+1])], 
           color='red', s=150, zorder=5, marker='*', label='この範囲の平均')

ax2.set_xlabel('日数')
ax2.set_ylabel('株価')
ax2.set_title('移動平均の計算：黄色の範囲の平均 → 赤い★の値', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n重要な気づき：")
print("- ウィンドウサイズが大きいほど、曲線が滑らかになる（高周波ノイズが除去される）")
print("- しかし、急激な変化への反応は遅くなる（トレードオフ）")
print("- この『周囲を見て値を決める』操作が、まさに畳み込みの本質です")

### 1.3 「周囲を見て自分を決める」操作

ここまでの例をまとめると、畳み込みの本質は：

> **「自分の周囲の値を集めて、何らかのルールで混ぜ合わせ、新しい自分の値を決める」**

この「ルール」を定義するのが**カーネル（kernel）**または**フィルタ（filter）**と呼ばれるものです。

In [ ]:
# カーネルの概念を図示

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 1. 平均カーネル（すべて同じ重み）
ax1 = axes[0]
avg_kernel = np.ones((3, 3)) / 9
im1 = ax1.imshow(avg_kernel, cmap='Blues', vmin=0, vmax=0.2)
for i in range(3):
    for j in range(3):
        ax1.text(j, i, f'{avg_kernel[i,j]:.2f}', ha='center', va='center', fontsize=12)
ax1.set_title('平均カーネル（ぼかし）\n全員に同じ重み', fontsize=11)
ax1.set_xticks([])
ax1.set_yticks([])

# 2. ガウシアンカーネル（中心が重い）
ax2 = axes[1]
gauss_kernel = np.array([[1, 2, 1],
                         [2, 4, 2],
                         [1, 2, 1]]) / 16
im2 = ax2.imshow(gauss_kernel, cmap='Oranges', vmin=0, vmax=0.3)
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{gauss_kernel[i,j]:.2f}', ha='center', va='center', fontsize=12)
ax2.set_title('ガウシアンカーネル\n中心ほど重要', fontsize=11)
ax2.set_xticks([])
ax2.set_yticks([])

# 3. エッジ検出カーネル（周囲と自分の差を見る）
ax3 = axes[2]
edge_kernel = np.array([[-1, -1, -1],
                        [-1,  8, -1],
                        [-1, -1, -1]])
im3 = ax3.imshow(edge_kernel, cmap='RdBu', vmin=-2, vmax=8)
for i in range(3):
    for j in range(3):
        ax3.text(j, i, f'{edge_kernel[i,j]:d}', ha='center', va='center', fontsize=12,
                color='white' if abs(edge_kernel[i,j]) > 2 else 'black')
ax3.set_title('エッジ検出カーネル\n自分と周囲の「差」を計算', fontsize=11)
ax3.set_xticks([])
ax3.set_yticks([])

plt.suptitle('カーネル = 「周囲をどう混ぜるか」のレシピ', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nカーネルの読み方：")
print("- 各マスの数字 = その位置の値にかける『重み』")
print("- 平均カーネル：全員を等しく見る → ぼかし効果")
print("- ガウシアン：近い点をより重視 → 自然なぼかし")
print("- エッジ検出：自分と周囲の差を強調 → 境界線が浮かび上がる")

---

## 2. 1次元信号での畳み込み入門

畳み込みの動作を、最もシンプルな1次元で詳しく見ていきましょう。

### 2.1 1次元畳み込みの基本操作

1次元畳み込みの手順：

1. 信号の上にカーネルを置く
2. カーネルの各要素と、その下にある信号の値を掛け算
3. 掛け算の結果をすべて足す
4. その合計が、「その位置」の出力値
5. カーネルを1つずらして繰り返す

In [ ]:
# 1次元畳み込みをステップバイステップで可視化

def visualize_1d_convolution_step(signal, kernel, position):
    """1次元畳み込みの1ステップを可視化"""
    k_size = len(kernel)
    pad = k_size // 2
    
    # パディングした信号
    padded_signal = np.pad(signal, pad, mode='constant', constant_values=0)
    
    # 現在の位置での計算
    window = padded_signal[position:position + k_size]
    products = window * kernel
    result = np.sum(products)
    
    return window, products, result, padded_signal

# サンプル信号とカーネル
signal = np.array([0, 0, 1, 2, 3, 2, 1, 0, 0, 0])
kernel = np.array([1, 2, 1]) / 4  # 簡単な平滑化カーネル

print("入力信号:", signal)
print("カーネル:", kernel)
print("\n--- 畳み込み計算のステップ ---\n")

# 各位置での計算を表示
output = []
for pos in range(len(signal)):
    window, products, result, _ = visualize_1d_convolution_step(signal, kernel, pos)
    output.append(result)
    print(f"位置 {pos}: 窓={window} × カーネル{kernel} = {products} → 合計 {result:.2f}")

print(f"\n出力信号: {np.array(output).round(2)}")

### 2.2 アニメーションで見る「滑らせながら掛ける」

In [ ]:
# 1次元畳み込みのアニメーション

def create_1d_conv_animation():
    """1次元畳み込みのアニメーションを作成"""
    # より長い信号
    t = np.linspace(0, 4*np.pi, 50)
    signal = np.sin(t) + 0.3 * np.sin(5*t)  # メイン信号 + 高周波ノイズ
    
    # 平滑化カーネル（5点移動平均）
    kernel = np.ones(5) / 5
    k_size = len(kernel)
    pad = k_size // 2
    
    # パディング
    padded = np.pad(signal, pad, mode='edge')
    
    # 出力を計算
    output = np.zeros_like(signal)
    for i in range(len(signal)):
        output[i] = np.sum(padded[i:i+k_size] * kernel)
    
    # アニメーション
    fig, axes = plt.subplots(3, 1, figsize=(12, 8))
    
    # 上段：入力信号
    ax1 = axes[0]
    ax1.plot(signal, 'b-', linewidth=2, label='入力信号')
    ax1.set_ylabel('入力')
    ax1.set_title('1次元畳み込み：カーネルを滑らせながら掛け算・足し算', fontsize=13, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    highlight1, = ax1.plot([], [], 'ro-', linewidth=3, markersize=8)
    
    # 中段：カーネルとの掛け算
    ax2 = axes[1]
    ax2.set_ylabel('重み付き値')
    ax2.set_ylim(-0.5, 0.5)
    ax2.grid(True, alpha=0.3)
    bars = ax2.bar(range(k_size), np.zeros(k_size), color='orange', alpha=0.7)
    ax2.axhline(y=0, color='black', linewidth=0.5)
    sum_text = ax2.text(k_size-1, 0.4, '', fontsize=12, ha='right')
    
    # 下段：出力信号
    ax3 = axes[2]
    ax3.plot(signal, 'b-', alpha=0.3, linewidth=1, label='入力（参考）')
    ax3.set_ylabel('出力')
    ax3.set_xlabel('位置')
    ax3.legend(loc='upper right')
    ax3.grid(True, alpha=0.3)
    output_line, = ax3.plot([], [], 'r-', linewidth=2, label='出力（畳み込み結果）')
    output_point, = ax3.plot([], [], 'ro', markersize=10)
    
    def init():
        highlight1.set_data([], [])
        output_line.set_data([], [])
        output_point.set_data([], [])
        return highlight1, output_line, output_point, sum_text, *bars
    
    def update(frame):
        pos = frame
        
        # ウィンドウ内の値をハイライト
        window_idx = range(max(0, pos-pad), min(len(signal), pos+pad+1))
        highlight1.set_data(window_idx, signal[list(window_idx)])
        
        # 掛け算の結果をバーで表示
        window = padded[pos:pos+k_size]
        products = window * kernel
        for bar, height in zip(bars, products):
            bar.set_height(height)
        
        # 合計を表示
        total = np.sum(products)
        sum_text.set_text(f'合計: {total:.3f}')
        
        # 出力を更新
        output_line.set_data(range(pos+1), output[:pos+1])
        output_point.set_data([pos], [output[pos]])
        
        return highlight1, output_line, output_point, sum_text, *bars
    
    anim = FuncAnimation(fig, update, init_func=init, 
                        frames=len(signal), interval=150, blit=True)
    plt.tight_layout()
    return anim

# アニメーション作成・表示
anim = create_1d_conv_animation()
HTML(anim.to_jshtml())

### 2.3 音声波形の平滑化例

In [ ]:
# 音声波形のようなノイズ除去の例

# 模擬的な「ノイズの乗った音声」を生成
duration = 0.1  # 秒
sample_rate = 4000
t = np.linspace(0, duration, int(sample_rate * duration))

# 基本波形（「あー」という声のような正弦波の組み合わせ）
clean_signal = (
    np.sin(2 * np.pi * 200 * t) +       # 基本周波数 200Hz
    0.5 * np.sin(2 * np.pi * 400 * t) + # 第2倍音
    0.3 * np.sin(2 * np.pi * 600 * t)   # 第3倍音
)

# ノイズを追加
noise = np.random.randn(len(t)) * 0.5
noisy_signal = clean_signal + noise

# 畳み込みで平滑化（移動平均）
def convolve_1d(signal, kernel):
    """1次元畳み込み（same padding）"""
    k_size = len(kernel)
    pad = k_size // 2
    padded = np.pad(signal, pad, mode='edge')
    output = np.zeros_like(signal)
    for i in range(len(signal)):
        output[i] = np.sum(padded[i:i+k_size] * kernel)
    return output

# 異なるサイズのカーネルで平滑化
kernel_3 = np.ones(3) / 3
kernel_7 = np.ones(7) / 7
kernel_15 = np.ones(15) / 15

smoothed_3 = convolve_1d(noisy_signal, kernel_3)
smoothed_7 = convolve_1d(noisy_signal, kernel_7)
smoothed_15 = convolve_1d(noisy_signal, kernel_15)

# 可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# 表示範囲（一部だけ拡大）
show_range = slice(50, 200)

axes[0, 0].plot(t[show_range]*1000, noisy_signal[show_range], 'gray', alpha=0.7)
axes[0, 0].plot(t[show_range]*1000, clean_signal[show_range], 'b-', linewidth=2)
axes[0, 0].set_title('元の信号（青）とノイズ入り信号（灰）')
axes[0, 0].set_xlabel('時間 (ms)')
axes[0, 0].legend(['ノイズ入り', '元の信号'], loc='upper right')

axes[0, 1].plot(t[show_range]*1000, noisy_signal[show_range], 'gray', alpha=0.3)
axes[0, 1].plot(t[show_range]*1000, smoothed_3[show_range], 'g-', linewidth=2)
axes[0, 1].set_title('3点移動平均で平滑化')
axes[0, 1].set_xlabel('時間 (ms)')

axes[1, 0].plot(t[show_range]*1000, noisy_signal[show_range], 'gray', alpha=0.3)
axes[1, 0].plot(t[show_range]*1000, smoothed_7[show_range], 'orange', linewidth=2)
axes[1, 0].set_title('7点移動平均で平滑化')
axes[1, 0].set_xlabel('時間 (ms)')

axes[1, 1].plot(t[show_range]*1000, noisy_signal[show_range], 'gray', alpha=0.3)
axes[1, 1].plot(t[show_range]*1000, smoothed_15[show_range], 'r-', linewidth=2)
axes[1, 1].plot(t[show_range]*1000, clean_signal[show_range], 'b--', alpha=0.5, linewidth=1)
axes[1, 1].set_title('15点移動平均で平滑化（青点線=元の信号）')
axes[1, 1].set_xlabel('時間 (ms)')

for ax in axes.flat:
    ax.grid(True, alpha=0.3)
    ax.set_ylabel('振幅')

plt.suptitle('畳み込みによるノイズ除去：カーネルサイズと平滑化の関係', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察ポイント：")
print("- カーネルが大きいほど、ノイズは減るが、元の波形も崩れる")
print("- 15点移動平均では、波形のピークが削られている（高周波成分の損失）")
print("- これは画像のぼかしでエッジが失われるのと同じ現象です")

---

## 3. 2次元への拡張：画像という信号

1次元の畳み込みを理解したら、2次元への拡張は直感的です。

**1次元 → 2次元の違い：**
- 1次元：左右の隣だけを見る
- 2次元：上下左右（＋斜め）の隣を見る

### 3.1 ピクセルの近傍という概念

In [ ]:
# 2次元畳み込みの概念図

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# サンプル画像（小さなグリッド）
sample_image = np.array([
    [10, 20, 30, 40, 50],
    [20, 30, 40, 50, 60],
    [30, 40, 50, 60, 70],
    [40, 50, 60, 70, 80],
    [50, 60, 70, 80, 90]
])

# 左：入力画像
ax1 = axes[0]
im1 = ax1.imshow(sample_image, cmap='Blues')
for i in range(5):
    for j in range(5):
        ax1.text(j, i, f'{sample_image[i,j]}', ha='center', va='center', fontsize=10)
ax1.set_title('入力画像（5×5）', fontsize=12)
ax1.set_xticks(range(5))
ax1.set_yticks(range(5))

# 中央の点とその近傍をハイライト
from matplotlib.patches import Rectangle
rect = Rectangle((0.5, 0.5), 3, 3, fill=False, edgecolor='red', linewidth=3)
ax1.add_patch(rect)
ax1.plot(2, 2, 'r*', markersize=20)

# 中央：カーネル
ax2 = axes[1]
kernel = np.array([
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
]) / 16
im2 = ax2.imshow(kernel, cmap='Oranges')
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{kernel[i,j]:.2f}', ha='center', va='center', fontsize=12)
ax2.set_title('カーネル（3×3）\nガウシアン重み', fontsize=12)
ax2.set_xticks(range(3))
ax2.set_yticks(range(3))

# 右：計算プロセス
ax3 = axes[2]
ax3.axis('off')

# 計算式を表示
calculation_text = """
【位置(2,2)での畳み込み計算】

  入力の3×3領域      カーネル
  ┌─────────────┐    ┌─────────────┐
  │ 30  40  50 │    │0.06 0.12 0.06│
  │ 40  50  60 │  × │0.12 0.25 0.12│
  │ 50  60  70 │    │0.06 0.12 0.06│
  └─────────────┘    └─────────────┘

= 30×0.06 + 40×0.12 + 50×0.06
+ 40×0.12 + 50×0.25 + 60×0.12
+ 50×0.06 + 60×0.12 + 70×0.06

= 1.88 + 4.80 + 3.13
+ 4.80 + 12.5 + 7.20
+ 3.13 + 7.20 + 4.38

= 50.0  ← 出力値
"""
ax3.text(0.1, 0.5, calculation_text, fontsize=10, family='monospace',
         verticalalignment='center', transform=ax3.transAxes)
ax3.set_title('計算プロセス', fontsize=12)

plt.suptitle('2次元畳み込み：「周囲の3×3を見て、カーネルで重み付けして足す」', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.2 2次元畳み込みのアニメーション

In [ ]:
# 2次元畳み込みのアニメーション

def create_2d_conv_animation():
    """2次元畳み込みのアニメーションを作成"""
    # 入力画像（チェッカーボードパターン）
    input_img = np.zeros((8, 8))
    input_img[2:6, 2:6] = 1  # 中央に白い四角
    
    # カーネル（平均化）
    kernel = np.ones((3, 3)) / 9
    k_size = 3
    pad = k_size // 2
    
    # パディング
    padded = np.pad(input_img, pad, mode='constant', constant_values=0)
    
    # 出力を計算
    output = np.zeros_like(input_img)
    for i in range(input_img.shape[0]):
        for j in range(input_img.shape[1]):
            output[i, j] = np.sum(padded[i:i+k_size, j:j+k_size] * kernel)
    
    # アニメーション
    fig, axes = plt.subplots(1, 3, figsize=(14, 5))
    
    # 左：入力画像
    ax1 = axes[0]
    ax1.imshow(input_img, cmap='gray', vmin=0, vmax=1)
    ax1.set_title('入力画像', fontsize=12)
    ax1.set_xticks(range(8))
    ax1.set_yticks(range(8))
    rect = Rectangle((-0.5, -0.5), 3, 3, fill=False, edgecolor='red', linewidth=3)
    ax1.add_patch(rect)
    
    # 中央：カーネル
    ax2 = axes[1]
    ax2.imshow(kernel, cmap='Oranges', vmin=0, vmax=0.2)
    for i in range(3):
        for j in range(3):
            ax2.text(j, i, f'{kernel[i,j]:.2f}', ha='center', va='center', fontsize=11)
    ax2.set_title('カーネル（3×3平均）', fontsize=12)
    ax2.set_xticks(range(3))
    ax2.set_yticks(range(3))
    
    # 右：出力画像
    ax3 = axes[2]
    output_display = np.zeros_like(input_img)
    im3 = ax3.imshow(output_display, cmap='gray', vmin=0, vmax=1)
    ax3.set_title('出力画像', fontsize=12)
    ax3.set_xticks(range(8))
    ax3.set_yticks(range(8))
    current_point, = ax3.plot([], [], 'ro', markersize=15)
    
    def init():
        return im3, rect, current_point
    
    def update(frame):
        i = frame // 8
        j = frame % 8
        
        # カーネル位置を更新
        rect.set_xy((j - 0.5 - pad, i - 0.5 - pad))
        
        # 出力を更新
        output_display = np.zeros_like(input_img)
        for fi in range(frame + 1):
            fi_i = fi // 8
            fi_j = fi % 8
            output_display[fi_i, fi_j] = output[fi_i, fi_j]
        im3.set_array(output_display)
        
        # 現在の点
        current_point.set_data([j], [i])
        
        return im3, rect, current_point
    
    anim = FuncAnimation(fig, update, init_func=init,
                        frames=64, interval=100, blit=True)
    
    plt.suptitle('2次元畳み込み：カーネルを画像全体にスライドさせる', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    return anim

anim_2d = create_2d_conv_animation()
HTML(anim_2d.to_jshtml())

### 3.3 実際の画像での畳み込み効果

In [ ]:
# 実際の画像（合成）での畳み込み効果

def create_test_image(size=100):
    """テスト用の合成画像を作成"""
    img = np.zeros((size, size))
    
    # 四角形
    img[20:40, 20:40] = 0.8
    
    # 円
    y, x = np.ogrid[:size, :size]
    center = (70, 70)
    radius = 15
    mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2
    img[mask] = 0.6
    
    # グラデーション三角形
    for i in range(30):
        img[60+i, 20:20+30-i] = 0.3 + 0.02 * i
    
    # ノイズを追加
    img += np.random.randn(size, size) * 0.05
    img = np.clip(img, 0, 1)
    
    return img

def convolve_2d(image, kernel):
    """2次元畳み込み（same padding）"""
    k_h, k_w = kernel.shape
    pad_h, pad_w = k_h // 2, k_w // 2
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
    
    output = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            output[i, j] = np.sum(padded[i:i+k_h, j:j+k_w] * kernel)
    
    return output

# テスト画像を作成
test_img = create_test_image()

# 各種カーネル
kernels = {
    '元の画像': None,
    'ぼかし（5×5平均）': np.ones((5, 5)) / 25,
    'ガウシアンぼかし': np.array([[1,2,1],[2,4,2],[1,2,1]]) / 16,
    'シャープ化': np.array([[0,-1,0],[-1,5,-1],[0,-1,0]]),
    '水平エッジ': np.array([[-1,-2,-1],[0,0,0],[1,2,1]]),
    '垂直エッジ': np.array([[-1,0,1],[-2,0,2],[-1,0,1]]),
}

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(14, 10))
axes = axes.flatten()

for ax, (name, kernel) in zip(axes, kernels.items()):
    if kernel is None:
        result = test_img
    else:
        result = convolve_2d(test_img, kernel)
    
    # エッジ検出の場合は絶対値を取って正規化
    if 'エッジ' in name:
        result = np.abs(result)
        result = result / result.max() if result.max() > 0 else result
    
    ax.imshow(result, cmap='gray', vmin=0, vmax=1)
    ax.set_title(name, fontsize=12)
    ax.axis('off')

plt.suptitle('様々なカーネルによる畳み込み効果', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n各フィルタの効果：")
print("- ぼかし：周囲の平均を取る → 画像が滑らかに")
print("- シャープ化：自分を強調し周囲を引く → 輪郭がくっきり")
print("- エッジ検出：隣接ピクセルとの『差』を計算 → 境界線だけが浮かび上がる")

---

## 4. まとめ：なぜこの操作が重要なのか

### 4.1 畳み込みの本質（3行まとめ）

```
1. 畳み込み = 「周囲を見て、自分の値を決める」操作
2. カーネル = 「周囲をどのように混ぜるか」のレシピ
3. カーネルを変えると、同じ操作で全く異なる効果（ぼかし、エッジ検出など）が得られる
```

### 4.2 CNNへの接続：学習するカーネル

ここまでは「人間が設計した」カーネル（Sobel、ガウシアンなど）を見てきました。

**CNNの革命的なアイデア：**

> 「カーネルの値を、データから自動的に学習させたらどうか？」

これがCNNの本質です：
- 人間がフィルタを設計する代わりに、ニューラルネットワークが最適なフィルタを学習
- 画像分類なら「分類に役立つ特徴を抽出するカーネル」が自動的に見つかる
- これにより、人間には思いつかないような高度な特徴抽出が可能に

In [ ]:
# CNNの学習済みカーネルの例（イメージ）

fig, axes = plt.subplots(2, 4, figsize=(14, 7))

# 人間が設計したカーネル
human_kernels = [
    ('水平エッジ', np.array([[-1,-2,-1],[0,0,0],[1,2,1]])),
    ('垂直エッジ', np.array([[-1,0,1],[-2,0,2],[-1,0,1]])),
    ('ぼかし', np.ones((3,3))/9),
    ('シャープ', np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])),
]

# CNNが学習したカーネル（模擬）
np.random.seed(123)
learned_kernels = [
    ('学習済み#1', np.array([[-0.2, 0.5, -0.3],[0.8, -0.1, 0.4],[-0.5, 0.2, 0.1]])),
    ('学習済み#2', np.array([[0.3, -0.7, 0.2],[0.1, 0.9, -0.4],[0.5, -0.2, 0.3]])),
    ('学習済み#3', np.array([[-0.6, 0.3, 0.1],[0.4, -0.8, 0.5],[0.2, 0.6, -0.3]])),
    ('学習済み#4', np.array([[0.1, 0.4, -0.5],[-0.3, 0.7, 0.2],[0.6, -0.1, -0.4]])),
]

# 上段：人間設計
for ax, (name, kernel) in zip(axes[0], human_kernels):
    im = ax.imshow(kernel, cmap='RdBu', vmin=-2, vmax=2)
    ax.set_title(f'人間設計:\n{name}', fontsize=10)
    ax.axis('off')

# 下段：学習済み
for ax, (name, kernel) in zip(axes[1], learned_kernels):
    im = ax.imshow(kernel, cmap='RdBu', vmin=-1, vmax=1)
    ax.set_title(f'CNN学習:\n{name}', fontsize=10)
    ax.axis('off')

axes[0, 0].set_ylabel('人間が設計', fontsize=12)
axes[1, 0].set_ylabel('CNNが学習', fontsize=12)

plt.suptitle('CNNの革命：カーネルを「学習」する', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nポイント：")
print("- 人間設計：直感的に解釈可能（水平、垂直など）")
print("- CNN学習：人間には解釈しにくいが、タスクに最適化されている")
print("- 次のノートブックで、この『学習されるカーネル』の数学を深掘りします")

### 4.3 3DGS/NeoVerseへの接続

畳み込みの「周囲を見て自分を決める」という考え方は、3D Gaussian Splattingにも通じます：

| 概念 | CNN | 3DGS |
|------|-----|------|
| 基本単位 | ピクセル | 3Dガウシアン点 |
| 影響範囲 | カーネルサイズ（3×3など） | ガウシアンの共分散（広がり） |
| 混合方法 | 重み付き和 | α-blending |
| 学習対象 | カーネルの重み | ガウシアンの位置・形状・色 |

**共通する思想：「局所的な情報を集約して、全体を構成する」**

---

## 確認クイズ

In [ ]:
# 理解度チェック

print("="*60)
print("確認クイズ：畳み込みの基礎理解")
print("="*60)

questions = [
    {
        "q": "Q1. 畳み込みの本質を一言で表すと？",
        "options": [
            "A) 画像を小さくする操作",
            "B) 周囲を見て自分の値を決める操作",
            "C) ピクセルを並び替える操作",
            "D) 色を変換する操作"
        ],
        "answer": "B"
    },
    {
        "q": "Q2. カーネル（フィルタ）の役割は？",
        "options": [
            "A) 画像のサイズを決める",
            "B) 周囲の値をどのように混ぜるかのレシピ",
            "C) 出力の色を決める",
            "D) 入力画像を保存する"
        ],
        "answer": "B"
    },
    {
        "q": "Q3. ぼかし（平均化）カーネルを適用すると何が起こる？",
        "options": [
            "A) エッジが強調される",
            "B) 画像が暗くなる",
            "C) 高周波成分（細かい変化）が減少する",
            "D) 画像が反転する"
        ],
        "answer": "C"
    },
    {
        "q": "Q4. CNNにおける畳み込みの革新的な点は？",
        "options": [
            "A) カーネルのサイズを自動決定する",
            "B) カーネルの値をデータから学習する",
            "C) 畳み込みを高速化する",
            "D) 画像を自動トリミングする"
        ],
        "answer": "B"
    }
]

for q in questions:
    print(f"\n{q['q']}")
    for opt in q['options']:
        print(f"  {opt}")

print("\n" + "="*60)
print("解答は下のセルを実行してください")
print("="*60)

In [ ]:
# 解答表示
print("解答：")
print("Q1: B - 周囲を見て自分の値を決める操作")
print("Q2: B - 周囲の値をどのように混ぜるかのレシピ")
print("Q3: C - 高周波成分（細かい変化）が減少する")
print("Q4: B - カーネルの値をデータから学習する")

---

## 次のステップ

このノートブックでは、畳み込みの**直感的な理解**を深めました。

次のノートブック **81. 畳み込みの数学的定義** では：

- 離散畳み込みの厳密な数式
- 相関 (Correlation) と畳み込み (Convolution) の違い
- パディングとストライドの数学
- 出力サイズの計算公式

を学びます。数式を理解することで、実装時のバグを防ぎ、より深いレベルでCNNを理解できるようになります。

---

## 参考文献・リソース

- [3Blue1Brown: But what is a convolution?](https://www.youtube.com/watch?v=KuXjwB4LzSA) - 視覚的に美しい畳み込みの解説
- [Image Kernels Explained Visually](https://setosa.io/ev/image-kernels/) - インタラクティブなカーネル可視化
- Goodfellow et al., "Deep Learning" Chapter 9 - CNNの理論的背景